In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = "PROD"


log = structlog.get_logger()

2024-10-17 13:39:11 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="126071012:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-17 13:39:12 [info     ] Loaded 5 items into vault.    
2024-10-17 13:39:13 [info     ] Initialized Clickhouse client.
2024-10-17 13:39:13 [info     ] Query success: blocks in 1.29s 2000 rows
2024-10-17 13:39:14 [info     ] Query success: logs in 2.48s 187138 rows
2024-10-17 13:39:15 [info     ] Query success: transactions in 2.95s 43972 rows


In [3]:
dataframes

{'blocks': shape: (1_012, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 1779886  ┆ 30000000  ┆ 0x         ┆ 14             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 4285527  ┆ 30000000  ┆ 0x         ┆ 27             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 6187833  ┆ 30000000  ┆ 0x         ┆ 30             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 516

In [4]:
# check if our blocks are in the range we want

log.info(f"Min block number: {dataframes['blocks']['number'].min()}")
log.info(f"Max block number: {dataframes['blocks']['number'].max()}")
log.info(
    f"Min block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].min())}"
)
log.info(
    f"Max block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].max())}"
)

2024-10-17 13:39:15 [info     ] Min block number: 126070000   
2024-10-17 13:39:15 [info     ] Max block number: 126071011   
2024-10-17 13:39:15 [info     ] Min block number: 2024-09-30 19:26:17
2024-10-17 13:39:15 [info     ] Max block number: 2024-09-30 19:59:59


In [5]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [6]:
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fee,l2_contrib_gas_fees_priority_fee,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,26]","decimal[*,26]","decimal[*,26]","decimal[*,17]","decimal[*,17]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0xbdd04d2a31d25fcc1c786dc293ce…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070925,126070925,1,247,247,1,336310,336310,0,3775,3775,0,0.0000009073641745200000,0.0000009073641745200000,0.0000000000000000000000,0.0000007621675821500000,0.0000001451965923700000,0.0000000038286540750000,0.0000000000000000000000,0.0000003922265821500000,0.0000003699410000000000,0.00226626500000000000000000,0.00116626500000000000000000,0.00110000000000000000000000,7.35748139400000000,0.00000000100000000,2024-09-30 23:57:07,2024-09-30 23:57:07,0µs,1,0.041667,1,1,0
"""0xbaee38cfa5c6853093fe55dc5fca…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070413,126070413,1,30,30,1,34420,34420,0,1600,1600,0,0.0000001771585664150000,0.0000001771585664150000,0.0000000000000000000000,0.0000001243869271600000,0.0000000527716392550000,0.0000000016227408000000,0.0000000000000000000000,0.0000000360307871600000,0.0000000883561400000000,0.00361379800000000000000000,0.00104679800000000000000000,0.00256700000000000000000000,6.30998172400000000,0.00000000100000000,2024-09-30 23:40:03,2024-09-30 23:40:03,0µs,1,0.041667,1,1,0
"""0xc32680901ed5d56ef7cb2d66b602…",10,"""op""","""2024-09-30""",2,2,0,2,2,0,126070587,126070602,16,42,43,2,1704474,1704474,0,16035,16035,0,0.0000045406829888970000,0.0000045406829888970000,0.0000000000000000000000,0.0000038663227236900000,0.0000006743602652070000,0.0000000162629054550000,0.0000000000000000000000,0.0000021618487236900000,0.0000017044740000000000,null,null,null,8.04508949520411599,0.00000000100000000,2024-09-30 23:45:51,2024-09-30 23:46:21,30s,1,0.041667,1,1,0
"""0xca2c520e50824998b860061ae487…",10,"""op""","""2024-09-30""",3,3,0,3,3,0,126070870,126070953,84,29,31,3,63000,63000,0,4800,4800,0,0.0000003363273309090000,0.0000003363273309090000,0.0000000000000000000000,0.0000001476817230000000,0.0000001886456079090000,0.0000000048682224000000,0.0000000000000000000000,0.0000000720817230000000,0.0000000756000000000000,0.00234415433333333333333333,0.00114415433333333333333333,0.00120000000000000000000000,7.51887665033333333,0.00000000100000000,2024-09-30 23:55:17,2024-09-30 23:58:03,2m 46s,1,0.041667,1,1,0
"""0x381d83f407292611ddeff4d05167…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070848,126070848,1,2,2,1,21000,21000,0,1600,1600,0,0.0000001119687074800000,0.0000001119687074800000,0.0000000000000000000000,0.0000000452876970000000,0.0000000666810104800000,0.0000000016227408000000,0.0000000000000000000000,0.0000000452876970000000,0.0000000000000000000000,0.00215655700000000000000000,0.00215655700000000000000000,0.00000000000000000000000000,7.97314549200000000,0.00000000100000000,2024-09-30 23:54:33,2024-09-30 23:54:33,0µs,1,0.041667,1,1,0


In [7]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-17 13:39:15 [info     ] Writing 1 partitions to BQ [2024-09-30 00:00:00 ... 2024-09-30 00:00:00]
2024-10-17 13:39:15 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.3Krows 288.9KB to BQ temp.daily_address_summary
